In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
from sklearn.model_split import train_test_split
import cv2

In [ ]:
def preprocess_images(image_paths, target_size=(224, 224)):

    processed_images = []
    for path in image_paths:
        # Read image
        img = cv2.imread(path)
        # Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Resize
        resized = cv2.resize(gray, target_size)
        # Normalize pixel values
        normalized = resized / 255.0
        processed_images.append(normalized)
    
    return np.array(processed_images)

In [ ]:
def create_denomination_model(input_shape, num_classes):

    model = models.Sequential([
        # Input layer
        layers.Input(shape=input_shape),
        
        # First convolutional block
        layers.Conv2D(32, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.MaxPooling2D((2, 2)),
        
        # Second convolutional block
        layers.Conv2D(64, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.MaxPooling2D((2, 2)),
        
        # Third convolutional block
        layers.Conv2D(128, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.MaxPooling2D((2, 2)),
        
        # Fourth convolutional block
        layers.Conv2D(256, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.MaxPooling2D((2, 2)),
        
        # Flatten and dense layers
        layers.Flatten(),
        layers.Dense(512),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Dropout(0.5),
        
        # Output layer
        layers.Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [ ]:
def create_authenticity_model(input_shape):

    model = models.Sequential([
        # Input layer
        layers.Input(shape=input_shape),
        
        # First convolutional block
        layers.Conv2D(32, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.MaxPooling2D((2, 2)),
        
        # Second convolutional block
        layers.Conv2D(64, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.MaxPooling2D((2, 2)),
        
        # Third convolutional block with residual connection
        layers.Conv2D(128, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.MaxPooling2D((2, 2)),
        
        # Flatten and dense layers
        layers.Flatten(),
        layers.Dense(256),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Dropout(0.5),
        
        # Output layer (binary classification)
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [ ]:
def train_models(train_images, train_denominations, train_authenticity,
                val_images, val_denominations, val_authenticity,
                input_shape, num_classes, epochs=50, batch_size=32):
    denom_model = create_denomination_model(input_shape, num_classes)
    denom_history = denom_model.fit(
        train_images, train_denominations,
        validation_data=(val_images, val_denominations),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
            tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
        ]
    )
    
    # Create and train authenticity model
    auth_model = create_authenticity_model(input_shape)
    auth_history = auth_model.fit(
        train_images, train_authenticity,
        validation_data=(val_images, val_authenticity),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
            tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
        ]
    )
    
    return denom_model, auth_model, denom_history, auth_history

In [ ]:

if __name__ == "__main__":
    image_paths = [...] 
    denomination_labels = [...]  
    authenticity_labels = [...]  

    processed_images = preprocess_images(image_paths)
    

    X_train, X_val, y_denom_train, y_denom_val, y_auth_train, y_auth_val = train_test_split(
        processed_images, denomination_labels, authenticity_labels,
        test_size=0.2, random_state=42
    )
    

    y_denom_train = tf.keras.utils.to_categorical(y_denom_train)
    y_denom_val = tf.keras.utils.to_categorical(y_denom_val)
    
    input_shape = (224, 224, 1) 
    num_classes = len(set(denomination_labels))
    
    denom_model, auth_model, denom_history, auth_history = train_models(
        X_train, y_denom_train, y_auth_train,
        X_val, y_denom_val, y_auth_val,
        input_shape, num_classes
    )